In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# **Importación de los datos de properati y AR**

In [6]:
data = pd.read_csv("../data/properatti.csv", sep = ",", low_memory=False) 
#data.head(3)
data.shape

(121220, 26)

In [7]:
#import AR.csv trayendo algunas de las columnas que tienen info de ubicación
columnas = ['num', 'place_name', 'place_name2','s20', 'Lat','Lon', 's', 'rnch', 'Pais','s0','s1','s2','s3','s4','s5','s6','s7','place_with_parent_names','fecha']
dataAR = pd.read_csv("../data/AR.tsv", header=None, names=columnas,  usecols=[1,2,4,5,8,17], sep = "\t", low_memory=False) 

#elimino la cadena de caracteres America/ y separo la columna place_with_parent_names en columnas, 
#renombro las columnas resultantes para coincidir con las columnas de data
dataAR["place_with_parent_names"] = dataAR["place_with_parent_names"].replace({'America/':''}, regex=True)
dataAR[["country_name", "state_name"]] = dataAR["place_with_parent_names"].str.split("/", expand=True)
dataAR["state_name"]=dataAR["state_name"].replace({'_':' '}, regex=True)
dataAR.drop("place_with_parent_names", axis=1, inplace=True)
dataAR.head(3)

,place_name,place_name2,Lat,Lon,Pais,country_name,state_name
0,Sierra del 15,Sierra del 15,-37.96613,-57.94670,AR,Argentina,Buenos Aires
1,Puesto 1 Napoleón Cué,Puesto 1 Napoleon Cue,-28.63333,-56.11667,AR,Argentina,Cordoba
2,Cañada del Zorro,Canada del Zorro,-32.16667,-59.50000,AR,Argentina,Cordoba
